In [1]:
import pandas as pd
from datasets import load_dataset

In [2]:
metadata_raw = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_Video_Games", trust_remote_code=True)

In [3]:
metadata_raw_df = metadata_raw['full'].to_pandas()

In [4]:
metadata_raw_df

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,Video Games,Dash 8-300 Professional Add-On,5.0,1,[Features Dash 8-300 and 8-Q300 ('Q' rollout l...,[The Dash 8-300 Professional Add-On lets you p...,None,"{'hi_res': [None], 'large': ['https://m.media-...","{'title': [], 'url': [], 'user_id': []}",Aerosoft,"[Video Games, PC, Games]","{""Pricing"": ""The strikethrough price is the Li...",B000FH0MHO,None,None,None
1,Video Games,Phantasmagoria: A Puzzle of Flesh,4.1,18,[Windows 95],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Sierra,"[Video Games, PC, Games]","{""Best Sellers Rank"": {""Video Games"": 137612, ...",B00069EVOG,None,None,None
2,Video Games,NBA 2K17 - Early Tip Off Edition - PlayStation 4,4.3,223,[The #1 rated NBA video game simulation series...,[Following the record-breaking launch of NBA 2...,58.0,{'hi_res': ['https://m.media-amazon.com/images...,{'title': ['NBA 2K17 - Kobe: Haters vs Players...,2K,"[Video Games, PlayStation 4, Games]","{""Release date"": ""September 16, 2016"", ""Best S...",B00Z9TLVK0,None,None,None
3,Video Games,Nintendo Selects: The Legend of Zelda Ocarina ...,4.9,22,[Authentic Nintendo Selects: The Legend of Zel...,[],37.42,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Amazon Renewed,"[Video Games, Legacy Systems, Nintendo Systems...","{""Best Sellers Rank"": {""Video Games"": 51019, ""...",B07SZJZV88,None,None,None
4,Video Games,Thrustmaster Elite Fitness Pack for Nintendo Wii,3.0,3,"[Includes (9) Total Accessories, Pedometer, Wi...",[The Thrustmaster Motion Plus Elite Fitness Pa...,None,"{'hi_res': [None, None, None, None, None, None...","{'title': [], 'url': [], 'user_id': []}",THRUSTMASTER,"[Video Games, Legacy Systems, Nintendo Systems...","{""Release date"": ""November 1, 2009"", ""Pricing""...",B002WH4ZJG,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137264,None,Story of Seasons: Pioneers Of Olive Town (Nint...,4.5,397,[A wild world of discovery - tame the wilderne...,"[Product Description, Inspired by Tales of you...",31.04,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Marvelous Europe,"[Video Games, Nintendo Switch, Games]","{""Release date"": ""March 26, 2021"", ""Best Selle...",B09XQJS4CZ,None,None,None
137265,Video Games,MotoGP 18 (PC DVD) UK IMPORT REGION FREE,4.0,1,[Brand new game engine - MotoGP18 has been reb...,[Become the champion of the 2018 MotoGP Season...,None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Milestone,"[Video Games, Game Genre of the Month]","{""Pricing"": ""The strikethrough price is the Li...",B07DGPTGNV,None,None,None
137266,Cell Phones & Accessories,Century Accessory Soft Silicone Protective Ski...,2.9,19,"[Easy access to all buttons, controls and port...",[This soft case cover will add a splash of col...,None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Century Accessory,"[Video Games, Legacy Systems, Xbox Systems, Xb...","{""Package Dimensions"": ""2.76 x 2.76 x 0.2 inch...",B00HUWCQBW,None,None,None
137267,None,Hasbro Interactive Mr. Potato Head Activity Pa...,3.9,5,[],"[Amazon.com, Everyone's favorite master-of-dis...",None,"{'hi_res': [None, 'https://m.media-amazon.com/...","{'title': [], 'url': [], 'user_id': []}",Hasbro,"[Video Games, PC, Games]","{""Release date"": ""July 24, 1999"", ""Best Seller...",B00002S9MH,None,None,None


In [5]:
with pd.option_context('display.max_colwidth', None):
    display(metadata_raw_df.sample(10))

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
2141,Video Games,Game Boy Color - Berry (Renewed),4.6,55,[Requires 2 AA Batteries],"[The Game Boy Color, is a handheld game console from Nintendo. It is the successor of the Game Boy. The Game Boy Color, as suggested by the name, features a color screen. It is slightly thicker and taller than the Game Boy Light, which is a redesigned Game Boy released in 1998. As with the original Game Boy, it has an 8-bit processor and the custom Zilog Z80 central processing unit.]",115.24,"{'hi_res': ['https://m.media-amazon.com/images/I/81dsrUiRvRL._SL1500_.jpg', 'https://m.media-amazon.com/images/I/81+XJW18OBL._SL1500_.jpg', 'https://m.media-amazon.com/images/I/818cP1irWpL._SL1500_.jpg', 'https://m.media-amazon.com/images/I/81dsrUiRvRL._SL1500_.jpg', 'https://m.media-amazon.com/images/I/81+XJW18OBL._SL1500_.jpg', 'https://m.media-amazon.com/images/I/818cP1irWpL._SL1500_.jpg'], 'large': ['https://m.media-amazon.com/images/I/31lFhxDTC6L.jpg', 'https://m.media-amazon.com/images/I/31jZZQ7LpeL.jpg', 'https://m.media-amazon.com/images/I/41tOlIjz8KL.jpg', 'https://m.media-amazon.com/images/I/31lFhxDTC6L.jpg', 'https://m.media-amazon.com/images/I/31jZZQ7LpeL.jpg', 'https://m.media-amazon.com/images/I/41tOlIjz8KL.jpg'], 'thumb': ['https://m.media-amazon.com/images/I/31lFhxDTC6L._SX38_SY50_CR,0,0,38,50_.jpg', 'https://m.media-amazon.com/images/I/31jZZQ7LpeL._SX38_SY50_CR,0,0,38,50_.jpg', 'https://m.media-amazon.com/images/I/41tOlIjz8KL._SX38_SY50_CR,0,0,38,50_.jpg', 'https://m.media-amazon.com/images/I/31lFhxDTC6L._SX38_SY50_CR,0,0,38,50_.jpg', 'https://m.media-amazon.com/images/I/31jZZQ7LpeL._SX38_SY50_CR,0,0,38,50_.jpg', 'https://m.media-amazon.com/images/I/41tOlIjz8KL._SX38_SY50_CR,0,0,38,50_.jpg'], 'variant': ['MAIN', 'PT01', 'PT02', 'PT03', 'PT04', 'PT05']}","{'title': [], 'url': [], 'user_id': []}",Amazon Renewed,"[Video Games, Legacy Systems, Nintendo Systems, Game Boy Systems, Game Boy Color, Accessories]","{""Best Sellers Rank"": {""Video Games"": 24751, ""Game Boy Color Accessories"": 59}, ""Pricing"": ""The strikethrough price is the List Price. Savings represents a discount off the List Price."", ""Product Dimensions"": ""8.3 x 5 x 2 inches; 8.78 Ounces"", ""Type of item"": ""Video Game"", ""Rated"": ""Everyone"", ""Is Discontinued By Manufacturer"": ""No"", ""Item Weight"": ""8.8 ounces"", ""Manufacturer"": ""Nintendo"", ""Date First Available"": ""August 26, 2019""}",B07X3MSKTK,None,None,None
135795,All Electronics,Ukor Fast Charging 4 x 2000mAh Rechargeable Battery Pack with Charger for Xbox Controller Rechargeable Batteries Xbox Accessories for Xbox One/Xbox Series X|S Xbox One S/Xbox One X/Xbox One Elite,4.4,224,"[【4 Pack 2000mAh Xbox Rechargeable Battery Pack】This cool and portable Ukor xbox controller battery pack comes with four 2000mAh rechargeable batteries, one charging cable, and one Xbox charging dock. It provides abundant power for Xbox One, Xbox One S, Xbox One X, and Xbox One Elite controllers, so you won't have to worry about interruptions in the middle of your game anymore., 【Non-Stop Game Fun】The Ukor Xbox battery pack rechargeable comes with four rechargeable batteries that can all be fully charged within 3 hours. Each battery is designed to be used up to 2000 times and can provide over 15 hours of gameplay time. With these four high-capacity rechargeable Xbox batteries, you can extend your gaming time significantly during family gatherings and game parties, creating unforgettable memories for your loved ones and friends., 【Safe Xbox Controller Battery】Our Xbox Series X controller battery pack comes with a built-in battery explosion-proof system, providing overvoltage, overcurrent, overheating, and short circuit protection. You can confidently charge your Xbox battery pack before going to sleep or heading out, and continue enjoying gaming when you wake up or co

# Build Feature Transformations

In [6]:
train_df = pd.read_parquet("../data/train.parquet")
val_df = pd.read_parquet("../data/val.parquet")

In [7]:
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline

In [8]:
# Custom function to reshape data from 1D to 2D
def reshape_2d_to_1d(X):
    return np.array(X).reshape(-1)

def flatten_string_array_col(X):
    return np.array(['\n'.join(x) for x in X])

def tfidf_pipeline_steps():
    steps = [
        ('impute', SimpleImputer(strategy='constant', fill_value='')),
        ('reshape', FunctionTransformer(reshape_2d_to_1d, validate=False)),
        ('tfidf', TfidfVectorizer(min_df=5, max_features=1000, ngram_range=(1, 2)))
    ]
    return steps

def description_pipeline_steps():
    steps = [
        ('flatten_string_array_col', FunctionTransformer(flatten_string_array_col, validate=False)),
        ('tfidf', TfidfVectorizer(min_df=5, max_features=1000, ngram_range=(1, 2)))
    ]
    return steps

def categories_pipeline_steps():
    def tokenizer(s):
        return s.split('\n')
    steps = [
        ('flatten_string_array_col', FunctionTransformer(flatten_string_array_col, validate=False)),
        ('tfidf', CountVectorizer(tokenizer=tokenizer, token_pattern=None))
    ]
    return steps

data = pd.Series(["from 14.99", "14.99", "price: 9.99", "20 dollars", "none"])
price_pattern = r'\b((?:\d+\.\d*)|(?:\d+))\b'
display(data.str.extract(price_pattern))

def price_parse_dtype(series, pattern):
    return series.str.extract(pattern).astype(float)

def price_pipeline_steps(price_pattern):
    steps = [
        ('extract_price', FunctionTransformer(price_parse_dtype, kw_args=dict(pattern=price_pattern), validate=False)),
        ('impute', SimpleImputer(strategy='constant', fill_value=0)),
        ('min_max_scale', MinMaxScaler())
    ]
    return steps

,0
0,14.99
1,14.99
2,9.99
3,20
4,NaN


In [9]:
tfm = [
    ('main_category', OneHotEncoder(handle_unknown='ignore'), ['main_category']),  # One-hot encoding for categorical field
    ('title', Pipeline(tfidf_pipeline_steps()), ['title']),  # TF-IDF vectorizer for text field
    ('description', Pipeline(description_pipeline_steps()), 'description'),  # TF-IDF vectorizer for another text field
    ('categories', Pipeline(categories_pipeline_steps()), 'categories'),  # Count Vectorizer for multi-label categorical
    ('price', Pipeline(price_pipeline_steps(price_pattern)), 'price')  # Normalizing price
]
cols = [x[0] for x in tfm]
cols

['main_category', 'title', 'description', 'categories', 'price']

In [10]:
train_full_df = pd.merge(train_df, metadata_raw_df[['parent_asin'] + cols], how='left', on='parent_asin')
# Drop duplicated item so that the Pipeline only fit the unique item features
fit_df = train_full_df.drop_duplicates(subset=['parent_asin'])
val_full_df = pd.merge(val_df, metadata_raw_df[['parent_asin'] + cols], how='left', on='parent_asin')
train_full_df

,user_id,parent_asin,rating,timestamp,main_category,title,description,categories,price
0,AEVPPTMG43C6GWSR7I2UGRQN7WFQ,B0863MT183,4.0,1613701986538,All Electronics,KIWI design VR Stand Compatible with Quest 2/R...,[],"[Video Games, PC, Virtual Reality, Headsets]",19.99
1,AEVPPTMG43C6GWSR7I2UGRQN7WFQ,B08P8P7686,5.0,1613702112995,All Electronics,Seltureone Accessories Compatible for Quest 2 ...,[],"[Video Games, PC, Virtual Reality, Headsets]",20.99
2,AEVPPTMG43C6GWSR7I2UGRQN7WFQ,B0B7LV3DN2,4.0,1617641445475,Cell Phones & Accessories,Eyglo Replacement Elite Strap for Oculus/Meta ...,[],"[Video Games, PC, Virtual Reality, Headsets]",28.49
3,AEVPPTMG43C6GWSR7I2UGRQN7WFQ,B09WMQ6DXG,5.0,1620231368468,All Electronics,"AMVR Controller Grips for Quest 2, Controller ...",[],"[Video Games, PC, Virtual Reality, Controllers]",15.99
4,AHV6QCNBJNSGLATP56JAWJ3C4G2A,B019WRM1IA,5.0,1451860309000,All Electronics,Microsoft Xbox 360 Wired Controller for Window...,"[Product Description, Precisely what you need ...","[Video Games, Legacy Systems, Xbox Systems, Xb...",67.83
...,...,...,...,...,...,...,...,...,...
18090,AHS2PQ33BWQLXC5NNUZS2BFXD34Q,B07TZT67KX,5.0,1622844181866,Video Games,Neverwinter Nights Enhanced Edition (Xbox One),"[Product Description, Slink through the shadow...","[Video Games, Xbox One, Games]",17.49
18091,AFO5SNKILFVJMSJJ2E3BRLDGE4NA,B09T5VN7D1,4.0,1601154352542,Video Games,13 Sentinels: Aegis Rim - PlayStation 4,"[Vanillaware, the storytellers behind Odin Sph...","[Video Games, PlayStation 4, Games]",23.06
18092,AFO5SNKILFVJMSJJ2E3BRLDGE4NA,B09918MSTF,5.0,1602615880364,Video Games,Story of Seasons: Friends of Mineral Town - Ni...,[Originally released for the Game Boy Advance ...,"[Video Games, Nintendo Switch, Games]",29.99
18093,AEFPHMM7CLX4UJNXJFQF4ZF5GNAA,B07P27XFP7,5.0,1599585146628,Video Games,Shin Megami Tensei: Strange Journey Redux - Ni...,"[""A spatial distortion full of demons appeared...","[Video Games, Legacy Systems, Nintendo Systems...",69.95


In [11]:
# Define preprocessing steps for each column
preprocessing_pipeline = ColumnTransformer(
    transformers=tfm,
    remainder='drop'  # Drop any columns not specified in transformers
)

# Create a pipeline object
item_metadata_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessing_pipeline)
])

# Fit the pipeline
item_metadata_pipeline.fit(fit_df)

# Transform the data (useful for training)
transformed_item_metadata = item_metadata_pipeline.transform(train_full_df)

# For demonstration, print the shape of the transformed features and a few rows
print(f"Transformed Item Metadata Shape: {transformed_item_metadata.shape}")

Transformed Item Metadata Shape: (18095, 2141)


# Persist

In [12]:
import joblib
import pickle

In [13]:
train_full_df.to_parquet("../data/train_item_features.parquet")
val_full_df.to_parquet("../data/val_item_features.parquet")

In [14]:
joblib.dump(item_metadata_pipeline, '../data/item_metadata_pipeline.joblib')

['../data/item_metadata_pipeline.joblib']